# Discussion Forum 69 case study on rice cultivation and preparation

Create a [conda](https://conda.io/miniconda.html) environment with the following:

    conda create -n df69 -c conda-forge -c cmutel -c haasad -c konstantinstadler python=3.6 bw2regional rower bw2_aware jupyter
    
Then activate it and run `jupyter notebook`.

You can see the [presentation given at the discussion forum here](https://github.com/cmutel/regionalized-lca-examples/raw/master/regionalization_df/DF%2069%20Mutel%20presentation.pdf).

In [ ]:
from bw2analyzer import ContributionAnalysis
from bw2regional.pandarus_remote import remote, AlreadyExists
import brightway2 as bw
import bw2_aware
import bw2regional as bwr
import os
import rower

In [ ]:
bw.projects.set_current("DF69 Case Study")

# Import ecoinvent 3.5

In [ ]:
bw.create_default_biosphere3()

In [ ]:
bw.create_core_migrations()

In [ ]:
ei = bw.SingleOutputEcospold2Importer(
    "/Users/cmutel/Downloads/ecoinvent 3.5_cutoff_ecoSpold02/datasets", # Change to path on your machine
    "ecoinvent 3.5 cutoff"
)
ei.apply_strategies()
ei.statistics()

In [ ]:
ei.write_database()

## Fix bug in brightway

We are incorrectly accounting for water balances. Need to set water production values to negative, water consumption to positive (because CFs, representing damage, are positive).

In [ ]:
water_production = [
    x for x in bw.Database("biosphere3")
    if 'water' in x['name'].lower() and x['categories'][0] == 'water'
]

In [ ]:
water_production_codes = [x['code'] for x in water_production]

In [ ]:
from bw2data.backends.peewee.proxies import *

In [ ]:
exc_qs = ExchangeDataset.select().where(
    ExchangeDataset.input_code << tuple(water_production_codes)
)

len(exc_qs)

In [ ]:
for exc in exc_qs:
    exc.data['amount'] *= -1
    exc.save()

In [ ]:
bw.Database("ecoinvent 3.5 cutoff").process()

# Import case study inventory

In [ ]:
imp = bw.ExcelImporter("rice_inventory.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location', 'reference product'))
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location'))
imp.match_database(fields=['name', 'location'])
imp.statistics()

In [ ]:
imp.write_database()

# Fix geographic data in LCI databases

## Install basic world definitions

In [ ]:
bwr.bw2regionalsetup()

## Define rest of world locations in ecoinvent

In [ ]:
rower_obj = rower.Rower('ecoinvent 3.5 cutoff')
rower_obj.load_existing(rower_obj.EI_3_5_CUTOFF)
rower_obj.label_RoWs()

## Label locations with their "geocollection"

Need to turn e.g. `"Europe without CH"` into `("ecoinvent", "Europe without CH")`.

In [ ]:
bwr.label_activity_geocollections("ecoinvent 3.5 cutoff")
bwr.label_activity_geocollections("AWARE Case Study")

In [ ]:
bw.databases['ecoinvent 3.5 cutoff']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['AWARE Case Study']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['biosphere3']['geocollections'] = []
bw.databases.flush()

# Import AWARE

In [ ]:
bw2_aware.import_aware()

# Add extension tables

## Define where rice is grown

Our rice inventory is country-specific, but this is too general for China and the US. We use data from Ramankutty on global rice production. Here is the map for part of Asia:

<img src="images/rice-asia.png">

In [ ]:
fp = os.path.abspath("rice.tiff")
assert os.path.isfile(fp)

bwr.geocollections['rice'] = {
    'filepath': fp,
    'band': 1
}

## Add population density

We assume that the spatial pattern of other water-consuming activities follows population density.

In [ ]:
fp = os.path.abspath("gdpweighted.tif")
assert os.path.isfile(fp)

bwr.geocollections['weighted-pop-density'] = {
    'filepath': fp,
    'band': 1
}

## Calculate raster statistics for our extension tables

In [ ]:
crop_rasters = [
    'rice',
    'weighted-pop-density',
]

for raster in crop_rasters:
    try:
        remote.calculate_rasterstats('world-topo-watersheds-aware', raster)
    except AlreadyExists:
        pass

In [ ]:
for raster in crop_rasters:
    remote.rasterstats_as_xt('world-topo-watersheds-aware', raster, "xt-" + raster)

# LCA calculations

We will split our calculations into the direct impact from irrigation, which will use the agricultural CFs, and everything else, which won't. We start by looking at the site-generic score and major contributors.

In [ ]:
swiss_rice = ('AWARE Case Study', 'Cooked rice CH')
chinese_rice = ('AWARE Case Study', 'Cooked rice CN')

In [ ]:
irrigation = [x for x in bw.Database("ecoinvent 3.5 cutoff") 
              if x['name'] == 'irrigation']

rice_production = [x for x in bw.Database("ecoinvent 3.5 cutoff") 
              if x['name'] == 'rice production']

## Rice in China

In [11]:
background_lca_cn = bwr.ExtensionTablesLCA(
    {chinese_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural'),
    xtable='xt-weighted-pop-density',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'exclude'
    }
)
background_lca_cn.lci()
background_lca_cn.lcia()
background_lca_cn.score

0.05157516184902011

In [12]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(background_lca_cn)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / background_lca_cn.score, amount])

['hard coal mine operation and hard coal preparation', 'kilogram', 'hard coal', 'CN', 0.011698903862484365, 0.22683213087593315, 0.245269991680576]
['water production and supply, decarbonised', 'kilogram', 'water, decarbonised, at user', ('RoW', 'RoW_213'), 0.009477375757022571, 0.18375852672583778, 1.0222104894900035]
['urea production, as N', 'kilogram', 'urea, as N', ('RoW', 'RoW_213'), 0.00891185756103606, 0.1727935936900095, 0.008734760321430033]
['tap water production, conventional treatment', 'kilogram', 'tap water', ('RoW', 'RoW_111'), 0.006497584943011585, 0.1259828318529074, 0.6573226615136141]
['treatment of wastewater from maize starch production, capacity 1.1E10l/year', 'cubic meter', 'wastewater from maize starch production', ('RoW', 'RoW_134'), -0.005556531270088297, -0.10773657456188607, 0.000682854535109166]
['ammonia production, steam reforming, liquid', 'kilogram', 'ammonia, liquid', ('RoW', 'RoW_213'), 0.004751021902367266, 0.0921184099484805, 0.00945616377796495]
[

In [13]:
foreground_lca_cn = bwr.ExtensionTablesLCA(
    {chinese_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
    xtable='xt-rice',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'include'
    }
)
foreground_lca_cn.lci()
foreground_lca_cn.lcia()
foreground_lca_cn.score

2.2740135085015747

In [14]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(foreground_lca_cn)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / foreground_lca_cn.score, amount])

['irrigation', 'cubic meter', 'irrigation', 'CN', 4.591251620185653, 2.019008067903249, 0.5886022828852308]
['rice production', 'kilogram', 'rice', 'CN', -2.418448601858576, -1.063515494880316, 0.9455359515231255]
['irrigation', 'cubic meter', 'irrigation', 'IN', 0.07924824468673732, 0.03484950480305488, 0.0032081108591706067]
['irrigation', 'cubic meter', 'irrigation', ('RoW', 'RoW_86'), 0.07681296313004095, 0.03377858699733743, 0.005649556260752386]
['rice production', 'kilogram', 'rice', 'IN', -0.04164134976012453, -0.018311830428643074, 0.0034612600957552976]
['rice production', 'kilogram', 'rice', ('RoW', 'RoW_155'), -0.024158196003775986, -0.010623593885198443, 0.006580082967391516]
['irrigation', 'cubic meter', 'irrigation', 'ES', 0.006878582510822316, 0.0030248644016872394, 8.49353257082777e-05]
['irrigation', 'cubic meter', 'irrigation', 'US', 0.0031189538504967553, 0.0013715634664597663, 0.00019420926940891196]
['irrigation', 'cubic meter', 'irrigation', 'PH', 0.0013551313664

## Rice in Switzerland

In [20]:
bwr.geocollections['world-topo-watersheds-aware']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/DF69-Case-Study.539531a9e1bbedcacdb6c969a8a2456f/regional/31371c7ad4d131f5b2685d8e20ad6d69bd17861030c9fb3b1feceab5990ed186.8bf6965994fc09a603386cd5ba02afd6e732d76ff02e4bb2a267c033239cfed3.geojson',
 'field': 'id',
 'url': 'https://pandarus.brightwaylca.org/intersection-file',
 'is intersection': True,
 'first': 'world',
 'second': 'watersheds-aware',
 'sha256': 'd565838cd79c2eb9b8d58ab3b22e1a14c405886894979616f22c33eb3198c91f',
 'kind': 'vector'}

In [15]:
background_lca_ch = bwr.ExtensionTablesLCA(
    {swiss_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural'),
    xtable='xt-weighted-pop-density',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'exclude'
    }
)
background_lca_ch.lci()
background_lca_ch.lcia()
background_lca_ch.score

0.05777860482058694

In [16]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(background_lca_ch)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / background_lca_ch.score, amount])

['water production and supply, decarbonised', 'kilogram', 'water, decarbonised, at user', ('RoW', 'RoW_213'), 0.00912998178126694, 0.1580166535626326, 0.9847412812294876]
['ammonia production, steam reforming, liquid', 'kilogram', 'ammonia, liquid', ('RoW', 'RoW_213'), 0.008054277304575266, 0.13939895796351018, 0.01603077545638354]
['treatment of wastewater from maize starch production, capacity 1.1E10l/year', 'cubic meter', 'wastewater from maize starch production', ('RoW', 'RoW_134'), -0.0055565396356836915, -0.09616950172019134, 0.0006828555631758638]
['urea production, as N', 'kilogram', 'urea, as N', ('RoW', 'RoW_213'), 0.0034811836810765273, 0.06025039358229979, 0.0034120052840635783]
['water production and supply, decarbonised', 'kilogram', 'water, decarbonised, at user', ('ecoinvent', 'RER'), 0.0033259456289968137, 0.05756361960148534, 0.39143903233936056]
['hard coal mine operation and hard coal preparation', 'kilogram', 'hard coal', 'CN', 0.002873593317975369, 0.0497345570544

In [17]:
foreground_lca_ch = bwr.ExtensionTablesLCA(
    {swiss_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
    xtable='xt-rice',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'include'
    }
)
foreground_lca_ch.lci()
foreground_lca_ch.lcia()
foreground_lca_ch.score

10.061722152468494

In [18]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(foreground_lca_ch)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / foreground_lca_ch.score, amount])

['irrigation', 'cubic meter', 'irrigation', 'US', 14.270410519593785, 1.4182870788270328, 0.8885819200993157]
['rice production', 'kilogram', 'rice', 'US', -4.353899917700955, -0.43271915599784183, 0.9411416720734768]
['irrigation', 'cubic meter', 'irrigation', 'IN', 0.10489357054681568, 0.010425011638895395, 0.004246279574501851]
['irrigation', 'cubic meter', 'irrigation', ('RoW', 'RoW_86'), 0.10167136039358228, 0.010104767240927908, 0.007477879348025468]
['rice production', 'kilogram', 'rice', 'IN', -0.05511753387113788, -0.00547794234783313, 0.004581410584997331]
['rice production', 'kilogram', 'rice', ('RoW', 'RoW_155'), -0.03197639351687837, -0.003178023904092148, 0.008709562680348015]
['irrigation', 'cubic meter', 'irrigation', 'CN', 0.029997201887246618, 0.002981318847080989, 0.0038456662739578956]
['rice production', 'kilogram', 'rice', 'CN', -0.01579667227350952, -0.0015699769914272617, 0.006175992964065163]
['irrigation', 'cubic meter', 'irrigation', 'ES', 0.00910443848124403